In [2]:
from datetime import date, timedelta
from joblib import Parallel, delayed
from functools import partial
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import argparse
import calendar
import datetime
import requests
import shapely
import pyproj
import shutil
import ee
import os

from helpers import *

In [3]:
shp_file = gpd.read_file("./all_boxes.shp")
shp_file

,shapeID,geometry
0,USA-ADM2-1590671490-B830,"POLYGON ((-83.90731 44.63817, -83.96620 44.647..."
1,USA-ADM2-1590671490-B830,"POLYGON ((-83.92066 44.59614, -83.97947 44.605..."
2,USA-ADM2-1590671490-B830,"POLYGON ((-83.93398 44.55414, -83.99272 44.563..."
3,USA-ADM2-1590671490-B830,"POLYGON ((-83.94728 44.51217, -84.00593 44.521..."
4,USA-ADM2-1590671490-B830,"POLYGON ((-83.92622 44.77398, -83.98540 44.783..."
...,...,...
381002,USA-ADM2-1590671490-B2001,"POLYGON ((-90.92832 47.57220, -90.99432 47.574..."
381003,USA-ADM2-1590671490-B2001,"POLYGON ((-90.93154 47.52756, -90.99744 47.529..."
381004,USA-ADM2-1590671490-B2001,"POLYGON ((-90.93476 47.48294, -91.00055 47.485..."
381005,USA-ADM2-1590671490-B2001,"POLYGON ((-90.95629 48.11253, -91.02354 48.114..."


In [4]:
ee.Initialize()

In [10]:
dates = GetDays('2010', '1')
dates

['2010-1-1', '2010-1-31']

In [6]:
cur_shp = ConvertToFeature(shp_file.geometry[0])
cur_shp

In [27]:
l5 = ee.ImageCollection("LANDSAT/LT05/C01/T1").filterDate(dates[0], dates[1]).filterBounds(cur_shp)
l5

In [28]:
m = ee.Algorithms.Landsat.simpleComposite(l5).select(['B3', 'B2', 'B1'])
m

In [29]:
l5.size().getInfo()

1

In [30]:
str(shp_file.geometry[0])

'POLYGON ((-83.90730607572486 44.63816518173967, -83.96620413216861 44.6476792811977, -83.95290501015911 44.6897496752692, -83.89392317475553 44.68022168977529, -83.90730607572486 44.63816518173967))'

In [31]:
shp_file.geometry[0].bounds

(-83.96620413216861,
 44.638165181739666,
 -83.89392317475553,
 44.689749675269205)

In [32]:
str(shp_file.geometry[0])

'POLYGON ((-83.90730607572486 44.63816518173967, -83.96620413216861 44.6476792811977, -83.95290501015911 44.6897496752692, -83.89392317475553 44.68022168977529, -83.90730607572486 44.63816518173967))'

In [33]:
geometry = ee.Geometry.Rectangle(list(shp_file.geometry[0].bounds))
# geometry

In [34]:
fname = "test.zip"

link = m.getDownloadURL({
        'name': "test",
        'crs': 'EPSG:4326',
        'fileFormat': 'GeoTIFF',
        'region': geometry,
        'scale':30
})

In [35]:
r = requests.get(link, allow_redirects=True)

open(fname, 'wb').write(r.content)

58470

In [ ]:
            fname = cur_directory + "/" + row.shapeID + "_" + str(year) + "_" + str(month) + ".zip"

            # Get the URL download link
            link = m.getDownloadURL({
                    'name': row.shapeID + "_" + str(year) + "_" + str(month),
                    'crs': 'EPSG:4326',
                    'fileFormat': 'GeoTIFF',
                    'region': geometry,
                    'scale':30
            })

            r = requests.get(link, allow_redirects=True)

            open(fname, 'wb').write(r.content)

In [ ]:
def main(year, ic, shp, month):

    ee.Initialize()

    # Get the start and end dates of each month in the year to filter the imagery
    dates = GetDays(year, month)

    # Set up imagery directories
    SetUp(year, month)

    # Make a new directory to organize the imagery
    cur_directory = os.path.join("./imagery", str(year), str(month))
    os.mkdir(cur_directory)

    # For the bounding box of every Mexico ADM2...
    for col, row in shp.iterrows():

        try:

            # Convert the ADM2 shape to a GEE feature
            cur_shp = ConvertToFeature(row.geometry)

            # Grab Landsat 5 Bands 1, 2 and 3 for the current month, year and ADM2
            l5 = ee.ImageCollection(ic).filterDate(dates[0], dates[1]).filterBounds(cur_shp)

            print(row.shapeName, " has ", l5.size().getInfo(), " images available in ", year)

            # Mosaic the images together using the min (using min to avoid the high values of clouds)
            m = ee.Algorithms.Landsat.simpleComposite(l5).select(['B3', 'B2', 'B1']);

            # Get the 4 point bounding box of the ADM2 to limit the export region
            geometry = ee.Geometry.Rectangle(list(row.geometry.bounds))

            fname = cur_directory + "/" + row.shapeID + "_" + str(year) + "_" + str(month) + ".zip"

            # Get the URL download link
            link = m.getDownloadURL({
                    'name': row.shapeID + "_" + str(year) + "_" + str(month),
                    'crs': 'EPSG:4326',
                    'fileFormat': 'GeoTIFF',
                    'region': geometry,
                    'scale':30
            })

            r = requests.get(link, allow_redirects=True)

            open(fname, 'wb').write(r.content)

        except:

            print("Imagery not available for ", row.shapeID, " during month ", str(month), " of ", str(year))




if __name__ == "__main__":

    # Trigger the authentication flow.
    # ee.Authenticate()

    # Parse input arguments
    parser = argparse.ArgumentParser(), default=["a", "b"]
    parser.add_argument("ic", help="GEE Imagery Collection ID")
    parser.add_argument("ISO", help="Country ISO3")
    parser.add_argument("--year_list", nargs="+")
    parser.add_argument("--month_list", nargs="+")
    args = parser.parse_args()# python3 DownloadImagery.py

    print(args.year_list)
    print(args.month_list)

    # Intitialize connection to GEE
    ee.Initialize()

    # Read in the shapefile to clip from
    shp_path = os.path.join("./data/", args.ISO, (args.ISO + "_bbox"), (args.ISO + "_bbox.shp"))
    shp = gpd.read_file(shp_path)

    # Parallelization
    num_cores = multiprocessing.cpu_count()
    output = Parallel(n_jobs=num_cores)(delayed(main)(year=y, ic=args.ic, shp=shp, month=j) for y in args.year_list for j in args.month_list)